In [1]:
# Import libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

import requests
from bs4 import BeautifulSoup

In [2]:
def clean_names(names):
    return names.str.lower().str.replace(' ', '').str.replace('-','')

## Top 100 mineable coins

In [3]:
# Top 100 mineable coins
url = 'https://coinmarketcap.com/coins/views/filter-non-mineable/'

r = requests.get(url) 

page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')
table = soup.find_all('div', class_='cmc-table__table-wrapper-outer')

In [4]:
# Get column names
column_names = []
for header in table[0].find_all('th'):
    column_names.append(str(header.string))
# Drop graphs
column_names.pop()
column_names.pop()

data = []
for row in table[2].find_all('tr')[1:]:
    new_row = []
    for entry in row:
        new_row.append(entry.text)
    new_row.pop()
    new_row.pop()
    data.append(new_row)
mineable_100 = pd.DataFrame(data, columns = column_names)
mineable_100 = mineable_100.set_index('Name')
mineable_100 = mineable_100.set_index(clean_names(mineable_100.index))
mineable_100 = mineable_100[['#', "Market Cap", "Price"]]
mineable_100 = mineable_100.rename(columns={'Market Cap': 'market_cap', 'Price':'price'})

## What to mine, nethash

### ASIC

In [5]:
url = 'https://whattomine.com/asic.json'

r = requests.get(url) 
data = r.json()

whattomine_asic = pd.DataFrame(data)
# normalize + clean names
whattomine_asic = pd.json_normalize(whattomine_asic['coins']).set_index(clean_names(whattomine_asic.index.str.lower()))
whattomine_asic['block_time'] = pd.to_numeric(whattomine_asic['block_time'])
whattomine_asic['nethash'] = pd.to_numeric(whattomine_asic['nethash'])
# Drop some columns
whattomine_asic = whattomine_asic[['tag','algorithm', 'block_time', 'difficulty', 'nethash', 'block_reward', 'exchange_rate', 'exchange_rate_curr']]
whattomine_asic[:4]

,tag,algorithm,block_time,difficulty,nethash,block_reward,exchange_rate,exchange_rate_curr
actinium,ACM,Lyra2z,151.0,3.470807e+03,9.872188e+10,50.0000,0.000001,BTC
axe,AXE,X11,158.0,8.481080e+05,2.305441e+13,1.9400,0.000040,BTC
bitcoin,BTC,SHA-256,564.0,1.391252e+13,1.059465e+20,12.6801,6454.160000,BTC
bitcoincashabc,BCH,SHA-256,616.0,4.441242e+11,3.096589e+18,12.5000,0.034112,BTC


### GPU

In [6]:
url = 'https://whattomine.com/coins.json'

r = requests.get(url) 
data = r.json()

whattomine_gpu = pd.DataFrame(data)
whattomine_gpu = pd.json_normalize(whattomine_gpu['coins']).set_index(clean_names(whattomine_gpu.index.str.lower()))
whattomine_gpu['block_time'] = pd.to_numeric(whattomine_gpu['block_time'])
whattomine_gpu['nethash'] = pd.to_numeric(whattomine_gpu['nethash'])
# Drop some columns
whattomine_gpu = whattomine_gpu[['tag','algorithm', 'block_time', 'difficulty', 'nethash', 'block_reward', 'exchange_rate', 'exchange_rate_curr']]
whattomine_gpu[:4]

,tag,algorithm,block_time,difficulty,nethash,block_reward,exchange_rate,exchange_rate_curr
aeternity,AE,CuckooCycle,177.0,3.186245e+07,180013,181.76400,1.511000e-05,BTC
aion,AION,"Equihash (210,9)",11.0,1.316311e+07,1196646,4.50000,9.010000e-06,BTC
anon,ANON,EquihashZero,150.0,3.468346e+05,2312,6.53125,8.600000e-07,BTC
beam,BEAM,BeamHashII,59.0,1.111586e+08,1884043,40.00000,4.158000e-05,BTC


## CoinWarz

In [7]:
url = 'https://www.coinwarz.com/v1/api/profitability?apikey=58467a6f2e2d4c65b7ef720b26137453&algo=all'
r = requests.get(url) 
data = r.json()

coinwarz = pd.DataFrame(data)
    
coinwarz = pd.json_normalize(coinwarz['Data'])
coinwarz = coinwarz.set_index(clean_names(coinwarz['CoinName']))

# Drop and rename some columns
coinwarz = coinwarz[['CoinTag', 'Algorithm','BlockTimeInSeconds', 'Difficulty', 'BlockReward', 'ExchangeRate']]
coinwarz = coinwarz.rename(columns={'CoinTag': 'tag', 'Algorithm': 'algorithm','BlockTimeInSeconds':'block_time',
                                    'Difficulty': 'difficulty', 'BlockReward': 'block_reward', 'ExchangeRate': 'exchange_rate'})
coinwarz['nethash_scraped'] = 0
coinwarz[:4]

,tag,algorithm,block_time,difficulty,block_reward,exchange_rate,nethash_scraped
CoinName,,,,,,,
cannabiscoin,CANN,X11,42,1.378400e+04,70.0,2.600000e-07,0
verge,XVG,Scrypt,30,7.220106e+03,730.0,3.800000e-07,0
ethereumclassic,ETC,EtHash,15,1.254596e+14,4.0,7.631100e-04,0
ethereum,ETH,EtHash,15,2.209444e+15,2.0,2.056421e-02,0


In [8]:
coinwarz.loc['bitcoin']

tag                        BTC
algorithm              SHA-256
block_time                 600
difficulty         1.39125e+13
block_reward              12.5
exchange_rate          6470.62
nethash_scraped              0
Name: bitcoin, dtype: object

Here, the exchange rate is relative the bitcoin, except for bitcoin itself, where it is in US$

# Combining the data

In [9]:
def check_missing(data):
    missing_data = result[data.isnull().any(axis=1)]
    return missing_data.shape[0]

In [10]:
whattomine = whattomine_gpu.append(whattomine_asic)
whattomine = whattomine.rename(columns={'nethash':'nethash_scraped'})
whattomine

,tag,algorithm,block_time,difficulty,nethash_scraped,block_reward,exchange_rate,exchange_rate_curr
aeternity,AE,CuckooCycle,177.0,3.186245e+07,1.800130e+05,181.76400,1.511000e-05,BTC
aion,AION,"Equihash (210,9)",11.0,1.316311e+07,1.196646e+06,4.50000,9.010000e-06,BTC
anon,ANON,EquihashZero,150.0,3.468346e+05,2.312000e+03,6.53125,8.600000e-07,BTC
beam,BEAM,BeamHashII,59.0,1.111586e+08,1.884043e+06,40.00000,4.158000e-05,BTC
bitcash,BITC,X25X,65.0,2.300000e+01,1.519758e+09,19.35000,6.800000e-07,BTC
...,...,...,...,...,...,...,...,...
vergegroestl,XVG,Myr-Groestl,150.0,3.869009e+06,1.107818e+14,400.00000,3.800000e-07,BTC
vergelyra2rev2,XVG,Lyra2REv2,150.0,1.539718e+05,4.408691e+12,400.00000,3.800000e-07,BTC
vergescrypt,XVG,Scrypt,150.0,7.220106e+03,2.067341e+11,400.00000,3.800000e-07,BTC
viacoin,VIA,Scrypt,23.0,2.924593e+05,5.461318e+13,0.62500,1.808000e-05,BTC


In [11]:
result = mineable_100.join(whattomine, how="left")

In [12]:
for coin in coinwarz.index:
    if coin not in whattomine.index.values:
        if coin in result.index:
            result.loc[coin, ['tag','algorithm','block_time','difficulty','nethash_scraped']] = coinwarz.loc[coin]
check_missing(result)

68

In [13]:
result[:10]

,#,market_cap,price,tag,algorithm,block_time,difficulty,nethash_scraped,block_reward,exchange_rate,exchange_rate_curr
Name,,,,,,,,,,,
bitcoin,1,"$118,527,913,943","$6,477.95",BTC,SHA-256,564.0000,1.391252e+13,1.059465e+20,12.680100,6454.160000,BTC
ethereum,2,"$14,721,614,335",$133.44,ETH,Ethash,13.3761,2.200812e+15,1.645332e+14,2.000000,0.020559,BTC
bitcoincash,3,"$4,061,701,553",$221.22,BCH,SHA-256,600.0000,4.466930e+11,0.000000e+00,NaN,NaN,NaN
bitcoinsv,4,"$3,066,491,895",$167.04,BSV,SHA-256,539.0000,3.787046e+11,3.017669e+18,12.500000,0.025808,BTC
litecoin,5,"$2,518,161,669",$39.10,LTC,Scrypt,144.0000,4.950637e+06,1.476585e+14,12.500000,0.006031,BTC
monero,6,"$826,168,510",$47.20,XMR,RandomX,118.0000,1.510894e+11,1.280419e+09,1.797923,0.007290,BTC
cardano,7,"$789,936,556",$0.030468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
tron,8,"$772,098,200",$0.011579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
dash,9,"$614,922,557",$65.31,DASH,X11,158.0000,1.778960e+08,4.835808e+15,1.553313,0.010068,BTC


In [42]:
result.to_csv('data/mineable_100.csv')